In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

## Lets get the data, model and setup training code

In [ ]:
train_loader = DataLoader(datasets.MNIST("./", train=True, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=True)
test_loader = DataLoader(datasets.MNIST("./", train=False, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=False)

print(f"Training images {len(train_loader.dataset)}, Test images {len(test_loader.dataset)}")

In [ ]:
class mnist_model(nn.Module):
  def __init__(self):
    super(mnist_model, self).__init__()
    self.layer1 = nn.Linear(784, 128, bias=True)
    self.layer2 = nn.Linear(128, 128, bias=True)
    self.layer3 = nn.Linear(128, 10, bias=True)
    # self.act = nn.Hardtanh()
    self.act = nn.ReLU()

  def forward(self, x):
    return self.layer3(self.act(self.layer2(self.act(self.layer1(x)))))

  def output(self, x):
    out1 = self.act(self.layer1(x))
    out2 = self.act(self.layer2(out1))
    out3 = self.layer3(out2)
    return out1, out2, out3

In [ ]:
model = mnist_model().cuda()
print(model)

epochs = 15
lr = 0.1

optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

## Training

In [ ]:
def get_acc(model, loader):
  correct = 0
  total = 0
  for img, label in loader:
    correct += torch.sum(torch.argmax(model(img.cuda().view(-1, 784)), -1).cpu() == label).item()
    total += len(img)
  return 100*correct/total

In [ ]:
for e in range(epochs):
  print("lr", optimizer.param_groups[0]["lr"])
  for img, label in train_loader:
    out = model(img.cuda().view(-1, 784))
    
    optimizer.zero_grad()
    loss = criterion(out, label.cuda())
    loss.backward()
    optimizer.step()
  lrs.step()
  print(f"Epoch {e}, training accuracy {get_acc(model, train_loader)}, test accuracy {get_acc(model, test_loader)}")

## Extract weights

In [ ]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [ ]:
for (name, p) in params:
  print(f"Layer {name.split('.')[0]}, type {name.split('.')[1]}, shape {p.shape}")

In [ ]:
# print(params)

In [ ]:
for img, label in train_loader:
  break

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Colab Notebooks/Falcon Neural Network/SecureML/"

In [ ]:
import os

np.savetxt(fname=path+"input_0", delimiter=" ", X=img.cuda().view(-1, 784).tolist())
np.savetxt(fname=path+"outputlayer1_0", delimiter=" ", X=model.output(img.cuda().view(-1, 784))[0].tolist())
np.savetxt(fname=path+"outputlayer2_0", delimiter=" ", X=model.output(img.cuda().view(-1, 784))[1].tolist())
np.savetxt(fname=path+"outputlayer3_0", delimiter=" ", X=model.output(img.cuda().view(-1, 784))[2].tolist())

np.savetxt(fname=path+"weight1_0", delimiter=" ", X=params[0][1].tolist())
np.savetxt(fname=path+"bias1_0", delimiter=" ",  X=params[1][1].tolist())
np.savetxt(fname=path+"weight2_0", delimiter=" ", X=params[2][1].tolist())
np.savetxt(fname=path+"bias2_0", delimiter=" ", X=params[3][1].tolist())
np.savetxt(fname=path+"weight3_0", delimiter=" ", X=params[4][1].tolist())
np.savetxt(fname=path+"bias3_0", delimiter=" ", X=params[5][1].tolist())